In [33]:
# simple_fuse: simple fusion from acoustic and txt
# run in MPC as : exec(open('simple_fuse_dev.py').read())
# 2019-12-20: initial work, it works!
# 2019-12-21: update to use silence feature from speech network
#             new data splitting (6000/2000/2039)

import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from calc_scores import calc_scores
from sklearn import linear_model
from sklearn.linear_model import Ridge,SGDRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel as c

# option: ser, ser_hfs, ser_ws, ter, ter_w2v, ter_glove, is saved change the name
# speech = 'ser'           # ser, ser_hfs, ser_ws
# text = 'ter'          # ter, ter_w2v, ter_glove'
ser = np.load('../speech/npy8000/speech_8000-npy10.npy')
ter = np.load('../text/npy8000/text_npy-glove-800019.npy')
mocap = np.load('../MOCAP/npy8000/mocap_npy-8000batch12.npy')
st = np.load('../speech+text/npy8000/speech_text_npy-8000glove6.npy')
sm = np.load('../speech+mocap/npy8000/speech_mocap_npy-8000batch15.npy')
tm = np.load('../text+mocap/npy8000/text_mocap_npy-8000batch19.npy')
stm = np.load('../speech+text+mocap/npy8000/stm_npy-glove-8000batch3.npy')
#mocap = np.load('../../data/predict_mocap_cnn1.npy')

In [34]:
# split dev and test
split = 1600
ser_dev = ser[:split]
ser_test = ser[split:]
ter_dev = ter[:split]
ter_test = ter[split:]
mocap_dev = mocap[:split]
mocap_test = mocap[split:]
st_dev = st[:split]
st_test = st[split:]
sm_dev = sm[:split]
sm_test = sm[split:]
tm_dev = tm[:split]
tm_test = tm[split:]
stm_dev = stm[:split]
stm_test = stm[split:]
# load label 
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# #remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) 
    scaled_vad = scaler.transform(vad) 
    vad = scaled_vad 
else:
    vad = vad

y_dev = vad[6400:8000]
y_test = vad[8000:] 

In [35]:
# SVR model
svr_rbf = SVR(kernel='rbf')
# svr_rbf = linear_model.RidgeCV(fit_intercept=True)

# svr_rbf = GaussianProcessRegressor(n_restarts_optimizer=10,alpha=0.1)
#svr_rbf = SVC(kernel='rbf', C=200, gamma=0.01,tol=0.01)
#svr_rbf = svm.SVC(c=0.8,kernel='linear',gamma=20,decision_function_shape='ovr')
# svr_rbf = SGDRegressor(loss="epsilon_insensitive")

# predicting valence
valence_model = svr_rbf.fit(np.array([ ser_dev[:,0],ter_dev[:,0],mocap_dev[:,0]]).T, y_dev[:,0])
valence_pred = valence_model.predict(np.array([ ser_test[:,0],ter_test[:,0],mocap_test[:,0]]).T)
# a_test = np.array([mocap_dev[:,0],ter_dev[:,0],ser_dev[:,0]]).T
ccc_v, pcc_v, rmse_v = calc_scores(valence_pred, y_test[:,0])

# predicting arousal
arousal_model = svr_rbf.fit(np.array([ ser_dev[:,1],ter_dev[:,1],mocap_dev[:,1]]).T, y_dev[:,1])
arousal_pred = arousal_model.predict(np.array([ ser_test[:,1],ter_test[:,1],mocap_test[:,1]]).T)
ccc_a, pcc_a, rmse_a = calc_scores(arousal_pred, y_test[:,1])

# predicting dominance
dominance_model = svr_rbf.fit(np.array([ser_dev[:,2],ter_dev[:,2],mocap_dev[:,2]]).T, y_dev[:,2])
dominance_pred = dominance_model.predict(np.array([ ser_test[:,2],ter_test[:,2],mocap_test[:,2]]).T)
ccc_d, pcc_d, rmse_d = calc_scores(dominance_pred, y_test[:,2])

vad_pred = np.vstack((valence_pred, arousal_pred, dominance_pred)).T
# filename = 'speech+text_mocap1.npy' 
# np.save(filename, vad_pred)

In [36]:
#SVR-S+T+M（late fusion）
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6989580177550961 0.6407102286687166 0.5220288085915626
PCC 0.7387457698432757 0.6946763736067436 0.5535990299818143
RMSE 0.27745343079239354 0.24796656383469584 0.29597414870887234


In [28]:
#SVR-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6899232919233831 0.6600395344562299 0.5153828306006422
PCC 0.735184564203424 0.7108784641990741 0.5443603072359916
RMSE 0.27984608651679294 0.24564971817309764 0.3034117988000121


In [4]:
#SVR-s+t
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5783908312449446 0.6571630682831308 0.5400872192844894
PCC 0.6474413682032755 0.7193055484532418 0.5754928155728917
RMSE 0.3163345334911346 0.24277449153539754 0.28754535843222234


In [18]:
#SVR-s+t+st
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5977939023272963 0.6571914281754212 0.5471046726483245
PCC 0.6796216287265199 0.728450180814621 0.5842771184371227
RMSE 0.3071943181066874 0.24614630956630199 0.2854561361606414


In [6]:
#SVR-s+m
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6021940985232673 0.6066691852739434 0.49502939118135847
PCC 0.6499954471604509 0.6500329142849738 0.5232960650873673
RMSE 0.3126264944958063 0.26478771700539533 0.30743648095153675


In [20]:
#SVR-s+m+sm
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.4632222453808155 0.6232037334219327 0.4845821539843707
PCC 0.5419798017885896 0.6705974948646489 0.5137975861239381
RMSE 0.3557800854539824 0.2624648557941767 0.3159802950349846


In [8]:
#SVR-t+m
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.684667337216424 0.40655033058930257 0.325571989819411
PCC 0.70632459282149 0.5160996034866683 0.41552536919040944
RMSE 0.29475178984889094 0.2912529403163279 0.3080418448717354


In [22]:
#SVR-t+m+tm
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.7085291355270946 0.417615435795206 0.33667963056054256
PCC 0.7331904147563093 0.5377753220343763 0.42687217849171655
RMSE 0.2805822485547428 0.28731246025301527 0.30566686291913203


In [10]:
#SVR-S+t+m
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.7111775897660199 0.6500482559453062 0.5243537923678822
PCC 0.7432590467436625 0.6989672179700941 0.5535305702722436
RMSE 0.27463685082440925 0.2477793471545164 0.2971780522359056


In [24]:
#SVR-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.7120670815527453 0.6536399944632169 0.5223154157339763
PCC 0.7466447203544485 0.6934943904172103 0.5530297672556432
RMSE 0.27295907735113095 0.2525010088778079 0.30173525052249495


In [23]:
# SVR model
#svr_rbf = SVR(kernel='rbf', C=200, gamma=0.1, epsilon=0.01)
#svr_rbf = linear_model.RidgeCV(fit_intercept=True)

#svr_rbf = GaussianProcessRegressor(n_restarts_optimizer=10,alpha=0.1)
#svr_rbf = SVC(kernel='rbf', C=200, gamma=0.01,tol=0.01)
#svr_rbf = svm.SVC(c=0.8,kernel='linear',gamma=20,decision_function_shape='ovr')
svr_rbf = SGDRegressor(loss="epsilon_insensitive")

# predicting valence
valence_model = svr_rbf.fit(np.array([ser_dev[:,0],ter_dev[:,0],mocap_dev[:,0],stm_dev[:,0]]).T, y_dev[:,0])
valence_pred = valence_model.predict(np.array([ ser_test[:,0],ter_test[:,0],mocap_test[:,0],stm_test[:,0]]).T)
# a_test = np.array([mocap_dev[:,0],ter_dev[:,0],ser_dev[:,0]]).T
ccc_v, pcc_v, rmse_v = calc_scores(valence_pred, y_test[:,0])

# predicting arousal
arousal_model = svr_rbf.fit(np.array([ ser_dev[:,1],ter_dev[:,1],mocap_dev[:,1],stm_dev[:,1]]).T, y_dev[:,1])
arousal_pred = arousal_model.predict(np.array([ ser_test[:,1],ter_test[:,1],mocap_test[:,1],stm_test[:,1]]).T)
ccc_a, pcc_a, rmse_a = calc_scores(arousal_pred, y_test[:,1])

# predicting dominance
dominance_model = svr_rbf.fit(np.array([ ser_dev[:,2],ter_dev[:,2],mocap_dev[:,2],stm_dev[:,2]]).T, y_dev[:,2])
dominance_pred = dominance_model.predict(np.array([ ser_test[:,2],ter_test[:,2],mocap_test[:,2],stm_test[:,2]]).T)
ccc_d, pcc_d, rmse_d = calc_scores(dominance_pred, y_test[:,2])

vad_pred = np.vstack((valence_pred, arousal_pred, dominance_pred)).T
# filename = 'speech+text_mocap1.npy' 
# np.save(filename, vad_pred)

In [5]:
#SGD-S+T
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5734623309354859 0.6566489648801255 0.5352494416379769
PCC 0.6506583412432885 0.7199367081119966 0.5937541777014231
RMSE 0.31376113490085944 0.24449383845050263 0.2799922329935011


In [11]:
#SGD-S+T+ST
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5973435736847986 0.6601262824109128 0.539273716661234
PCC 0.6885877402850057 0.7309054742198183 0.6104863535665129
RMSE 0.30506145586771255 0.24507663527455242 0.2754438288100449


In [13]:
#SGD-S+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5869765907305193 0.646697731157731 0.4966391687502096
PCC 0.6503125412991527 0.7061429585780022 0.539893111137743
RMSE 0.3118980177060406 0.24544954106069514 0.29267647088054805


In [15]:
#SGD-S+M+SM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6082440239354886 0.6495297320869272 0.502902018681501
PCC 0.6632038584241037 0.7097084830438462 0.5485465775751337
RMSE 0.30738279476454305 0.2467307900760675 0.2915948078893391


In [17]:
#SGD-T+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6671960912653089 0.412665368061878 0.33128952682972435
PCC 0.7168002550219846 0.5257084408472168 0.4341915215156561
RMSE 0.29108425116144715 0.2894867564866357 0.3045007346415027


In [20]:
#SGD-T+M+TM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6943038837302663 0.4294501376669898 0.3632483956550269
PCC 0.736719572586395 0.5452950628086746 0.4554669208576907
RMSE 0.2795351213241168 0.28554844011073643 0.30108612674589963


In [22]:
#SGD-S+T+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6841191275426102 0.657430212350735 0.5254600797486565
PCC 0.7305867246866732 0.7187653599221372 0.5730695131076381
RMSE 0.2815204216677186 0.2427511884179858 0.28529364952337577


In [24]:
#SGD-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.7265751103867627 0.6796037645076806 0.538912383133168
PCC 0.7645570122222342 0.7327935902461953 0.5802597308469843
RMSE 0.2644750951427117 0.2382221845466015 0.2822637376002736
